In [19]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
import $ivy.`org.apache.spark::spark-mllib:2.3.0`
import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import $exclude.$                        , $ivy.$                            // for cleaner logs

import $profile.$           

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                    

import $ivy.$                                   

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

In [20]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.ml.feature.Tokenizer
import jupyter.spark.session._

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.ml.feature.Tokenizer

import jupyter.spark.session._

cmd18.sc:1: object ml is not a member of package org.apache.spark
import org.apache.spark.ml.feature.{HashingTF, IDF, Tokenizer}
                        ^

: 

In [21]:
val sparkSession = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
  .jupyter() // this method must be called straightaway after builder()
  // .yarn("/etc/hadoop/conf") // optional, for Spark on YARN - argument is the Hadoop conf directory
  // .emr("2.6.4") // on AWS ElasticMapReduce, this adds aws-related to the spark jar list
  .master("local") // change to "yarn-client" on YARN
  // .config("spark.executor.instances", "10")
  // .config("spark.executor.memory", "3g")
  // .config("spark.hadoop.fs.s3a.access.key", awsCredentials._1)
  // .config("spark.hadoop.fs.s3a.secret.key", awsCredentials._2)
  .appName("notebook")
  .getOrCreate()

sparkSession: SparkSession = org.apache.spark.sql.SparkSession@3d1fa21d

In [22]:
sparkSession

res21: SparkSession = org.apache.spark.sql.SparkSession@3d1fa21d

## Load training dataset

In [23]:
var train = sparkSession.read.parquet("/home/mahjoubi/Documents/github/toxic_comment/data/parquet/train")

train: DataFrame = [id: string, comment_text: string ... 6 more fields]

In [24]:
train.count()

res23: Long = 159571L

In [25]:
train.printSchema()

root
 |-- id: string (nullable = true)
 |-- comment_text: string (nullable = true)
 |-- toxic: long (nullable = true)
 |-- severe_toxic: long (nullable = true)
 |-- obscene: long (nullable = true)
 |-- threat: long (nullable = true)
 |-- insult: long (nullable = true)
 |-- identity_hate: long (nullable = true)



In [26]:
train.select("id", "comment_text").show(5, truncate=true)

+----------------+--------------------+
|              id|        comment_text|
+----------------+--------------------+
|9f14426bcd93c0f8|"

 Edit warring ...|
|9f15fa8011f1283e|I cite a scholarl...|
|9f1624d262a42d8c|"

At the time I ...|
|9f163747e5cac904|(UTC)
The 2008 ra...|
|9f1638a5017abf77|Faryl Smith
Hi th...|
+----------------+--------------------+
only showing top 5 rows



In [27]:
train.drop("id", "comment_text").show(5, truncate=true)

+-----+------------+-------+------+------+-------------+
|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+-----+------------+-------+------+------+-------------+
|    0|           0|      0|     0|     0|            0|
|    0|           0|      0|     0|     0|            0|
|    0|           0|      0|     0|     0|            0|
|    0|           0|      0|     0|     0|            0|
|    0|           0|      0|     0|     0|            0|
+-----+------------+-------+------+------+-------------+
only showing top 5 rows



In [28]:


// val sentenceData = spark.createDataFrame(Seq(
//   (0.0, "Hi I heard about Spark"),
//   (0.0, "I wish Java could use case classes"),
//   (1.0, "Logistic regression models are neat")
// )).toDF("label", "sentence")

val tokenizer = new Tokenizer().setInputCol("comment_text").setOutputCol("words")
train = tokenizer.transform(train)

// val hashingTF = new HashingTF()
//   .setInputCol("words").setOutputCol("rawFeatures").setNumFeatures(20)

// val featurizedData = hashingTF.transform(wordsData)
// // alternatively, CountVectorizer can also be used to get term frequency vectors

// val idf = new IDF().setInputCol("rawFeatures").setOutputCol("features")
// val idfModel = idf.fit(featurizedData)

// val rescaledData = idfModel.transform(featurizedData)
// rescaledData.select("label", "features").show()

tokenizer: Tokenizer = tok_b6e3aca2e60e

In [30]:
train.select("id", "words").show()

+----------------+--------------------+
|              id|               words|
+----------------+--------------------+
|9f14426bcd93c0f8|[", , , edit, war...|
|9f15fa8011f1283e|[i, cite, a, scho...|
|9f1624d262a42d8c|[", , at, the, ti...|
|9f163747e5cac904|[(utc), the, 2008...|
|9f1638a5017abf77|[faryl, smith, hi...|
|9f1651ae5c97daa2|[why?, , , why, i...|
|9f181c696a35ef48|[", , oh, yeah., ...|
|9f1937cf6a27ba0b|[", , , blocked, ...|
|9f1ab38cb1f08a9b|[well,, you, shou...|
|9f1b88e97ec769a0|[please, refrain,...|
|9f1b9255a8d920b0|[i, corrected, th...|
|9f1c59e675fa5ab3|[hi, hello,, i, a...|
|9f1c62979ca058e1|[your, ga, nomina...|
|9f1cacc1d7a3baaf|[request, to, edi...|
|9f1ced25d7a385a7|["''i, added, thi...|
|9f1ee30afc7d9a4f|[", , i, don't, t...|
|9f2087bb5b38e928|[still, a, bit, c...|
|9f20c9142df40cf9|[march, 2011, (ut...|
|9f22af9d902a4c2b|[home, of, the, c...|
|9f23afd98fadd536|[vandalism, remov...|
+----------------+--------------------+
only showing top 20 rows

